In [1]:
import os
from os.path import join
import re
import numpy as np
import shutil
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from glob import glob
from scipy.spatial.transform import Rotation
from exr import read_exr

In [2]:
base_dir = '/bigpen/simulator_data/LEC_fast_result'

In [3]:
test_str = "Frame 00001 Rotation: X=-0.0003235038, Y=0.01255963, Z=3.553312E-05, W=0.9999211"
def parse_sim_poses(line: str):
    parsed = line.replace(',','').split(' ')
    poses = [float(parsed[1])] + [float(p[2:]) for p in parsed[3:] if len(p)>2]
    return poses
print(parse_sim_poses(test_str))

[1.0, -0.0003235038, 0.01255963, 3.553312e-05, 0.9999211]


In [4]:
%matplotlib widget

start = 120
skeletons = np.load("/playpen/skeleton-on-surface/figures/LEC_fast/skeletons.npy").astype(int)

positions = []
with open('/bigpen/simulator_data/LEC_fast/LEC_Camera Position Data.txt', 'r') as f:
    for line in f:
        if line[0]=='#' or len(line)<2:
            continue
        parsed = line.split(' ')
        positions.append(parse_sim_poses(line))
quaternions = []
with open('/bigpen/simulator_data/LEC_fast/LEC_Camera Quaternion Rotation Data.txt', 'r') as f:
    for line in f:
        if line[0]=='#' or len(line)<2:
            continue
        parsed = line.split(' ')
        quaternions.append(parse_sim_poses(line))

# Convert parsed data to numpy arrays and select only the frames that were skeletonized
pos_arr = np.array(positions)[:,1:]     # remove first column since it's just the frame numbers
quat_arr = np.array(quaternions)[:,1:]
pos_arr = pos_arr[start:start+len(skeletons)]
quat_arr = quat_arr[start:start+len(skeletons)]
rot_mat_arr = Rotation.from_quat(quat_arr).as_matrix() # type: np.ndarray

step = 1
skeletons = skeletons[::step]
pos_arr = pos_arr[::step]
rot_mat_arr = rot_mat_arr[::step]

In [5]:
# x is left, y is down in camera coords
cx = skeletons.shape[-1]//2
cy = skeletons.shape[-2]//2
fx = 144.2398/135*cx*2
fy = 144.2398/108*cy*2
camera_mat = np.array(
    [[fx, 0, cx],
     [0, fy, cy],
     [0, 0, 1]]
).astype(float)
inv_camera_mat = np.linalg.inv(camera_mat)

In [33]:
# Compute inverse pose matrix
Rt_inv = np.zeros((len(skeletons), 4, 4))
Rt_inv[:,:3,:3] = rot_mat_arr.transpose(0, 2, 1)
Rt_inv[:,:3,3] = -pos_arr
Rt_inv[:,3,3] = 1

Rt= np.zeros((len(skeletons), 4, 4))
Rt[:,:3,:3] = rot_mat_arr
Rt[:,:3,3] = pos_arr
Rt[:,3,3] = 1

points = []
frame_inds = []
rng = np.random.default_rng()
for i, sk in enumerate(skeletons):
    coords = np.vstack(np.where(sk>0))

    # Pick some points at random
    rng.shuffle(coords, axis=1)
    coords = coords[:,:100] if coords.shape[1]>=50 else coords
    coords = coords.astype(int)

    # coords = np.array([[sk.shape[1]//4, sk.shape[1]//4, 3*sk.shape[1]//4, 3*sk.shape[1]//4],
    #                   [sk.shape[0]//4, 3*sk.shape[0]//4, sk.shape[0]//4, 3*sk.shape[0]//4]])
    coords[1] = sk.shape[1] - coords[1]

    # Read in depths
    depths = read_exr(f"/bigpen/simulator_data/LEC_fast/LEC_depth{step*i+120:05}.exr")
    zs = depths[coords[0], coords[1]]
    # zs = np.ones_like(coords[1]).astype(float)*0.7

    # Transform coords to world coordinates
    # coords = np.vstack([coords, np.ones_like(zs)])
    # coords =  inv_camera_mat @ coords
    coords = coords.astype(float)
    coords -= np.array([[cx], [cy]])
    coords[0] *= 1/fx
    coords[1] *= 1/fy
    coords = coords*zs
    coords = np.vstack([coords, zs, np.ones_like(zs)])
    coords = Rt[i] @ coords
    points.append(coords)
    frame_inds.extend([i for j in range(coords.shape[1])])

points = np.hstack(points)
points = points[:3]

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.set_box_aspect([np.ptp(r) for r in np.hstack([points, pos_arr.T])])
# ax.set_xlabel('X')
# ax.set_zlabel('Z')
# ax.plot(*pos_arr.T)
# # ax.scatter(*old_pose_arr.T[1:4], c=colors)
# ax.scatter(*points)

# ax.quiver(*pos_arr.T, *(rot_mat_arr[:,:,0].T), color='blue', length=0.3)
# ax.quiver(*pos_arr.T, *(rot_mat_arr[:,:,1].T), color='red', length=0.1)
# ax.quiver(*pos_arr.T, *(rot_mat_arr[:,:,2].T), color='green', length=0.3)

In [39]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

coords = ['x', 'y', 'z']
fig = px.line_3d(pd.DataFrame({coords[i]:pos_arr[:,i] for i in range(3)}), x='x', y='y', z='z', labels='trajectory')
fig.add_trace(go.Scatter3d(
    x=pos_arr[:,0],
    y=pos_arr[:,1],
    z=pos_arr[:,2],
    marker={
        'color': np.arange(len(pos_arr)),
        # 'colorscale': 'viridis'
    },
    hovertemplate='<b>%{text}</b>',
    text=np.arange(len(pos_arr))
))
fig.add_trace(go.Scatter3d(
        x=points[0], y=points[1], z=points[2], mode='markers', name='skeleton points', marker={
            'size': 2,
            'color': frame_inds
        },
        hovertemplate='<b>%{text}</b>',
        text=frame_inds
    ))
fig.write_html('figures/points.html', auto_open=True)